In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import requests  # Thư viện để thực hiện các yêu cầu HTTP
from bs4 import BeautifulSoup  # Thư viện để phân tích cú pháp HTML
import csv  # Thư viện để làm việc với tệp CSV
import pandas as pd  # Thư viện để làm việc với DataFrame

# Đường dẫn đến tệp CSV chứa danh sách mã sinh viên
csv_file_path = '/content/drive/My Drive/udp_exam_scores.csv'

# Đọc tệp CSV để lấy danh sách mã sinh viên
df = pd.read_csv(csv_file_path)
display(df)
# Lấy danh sách mã sinh viên từ cột 'Mã sinh viên' và chuyển thành danh sách
student_ids = df['Mã sinh viên'].tolist()
# student_ids

,STT,Họ tên,Mã sinh viên,Lớp,Nhóm môn học,Ca thi,Điểm thi
0,1,Nguyễn Tuấn Anh,B21DCCN008,D21CNPM4,5,13/10/2024 08:00 - 09:00,......
1,2,Phạm Viết Tài Anh,B21DCCN012,D21HTTT6,6,11/10/2024 14:10 - 15:10,......
2,3,Tạ Xuân Bách,B21DCCN015,E21CNPM1,1,13/10/2024 13:00 - 14:00,......
3,4,Trần Công Bách,B21DCCN016,E21CNPM1,1,13/10/2024 13:00 - 14:00,......
4,5,Tạ Văn Chiến,B21DCCN021,D21HTTT4,4,11/10/2024 13:00 - 14:00,......
...,...,...,...,...,...,...,...
432,433,Nguyễn Quốc Vương,B21DCCN804,D21HTTT6,3,13/10/2024 15:20 - 16:20,......
433,434,Ngô Thị Xuân,B21DCCN806,D21CNPM2,3,13/10/2024 15:20 - 16:20,......
434,435,Lương Ngọc Yên,B21DCCN809,D21CNPM3,5,13/10/2024 08:00 - 09:00,......
435,436,Nguyễn Thế Mạnh,B21DCCN812,E21CNPM4,4,13/10/2024 14:10 - 15:10,......


In [7]:
base_url = 'http://203.162.10.109/search?txtKey='

def fetch_student_scores(student_id):
    """
    Hàm cào điểm thi cho sinh viên dựa trên mã sinh viên.
    """
    url = f'{base_url}{student_id}'  # Tạo URL truy cập
    response = requests.get(url)  # Thực hiện yêu cầu GET

    # Kiểm tra mã trạng thái của phản hồi
    if response.status_code != 200:
        print(f'Lỗi khi truy cập {url}: {response.status_code}')  # In thông báo lỗi nếu không truy cập được
        return None  # Trả về None nếu không thành công

    # Phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr')[1:]  # Lấy tất cả các hàng trừ hàng tiêu đề

    # Khởi tạo thông tin cho sinh viên
    student_info = {
        'name': '',
        'class': '',
        'subject_group': '',
        'tcp_score': '',
        'udp_score': ''
    }

    # Duyệt qua từng hàng để lấy thông tin
    for row in rows:
        # Lấy các ô tương ứng với tên, lớp, nhóm môn học, ghi chú và kết quả
        name_cell = row.find('td', {'data-label': 'Họ tên'})
        class_cell = row.find('td', {'data-label': 'Lớp'})
        subject_group_cell = row.find('td', {'data-label': 'Nhóm môn học'})
        subject_cell = row.find('td', {'data-label': 'Ghi chú'})
        score_cell = row.find('td', {'data-label': 'Kết quả'})

        # Cập nhật thông tin từ các ô
        student_info['name'] = name_cell.text.strip()  # Lưu tên sinh viên
        student_info['class'] = class_cell.text.strip()  # Lưu lớp
        student_info['subject_group'] = subject_group_cell.text.strip()  # Lưu nhóm môn học

        # Lưu điểm vào biến tương ứng
        if 'TCP' in subject_cell.text.strip():  # Nếu là môn TCP
            student_info['tcp_score'] = score_cell.text.strip()  # Lưu điểm TCP
        elif 'UDP' in subject_cell.text.strip():  # Nếu là môn UDP
            student_info['udp_score'] = score_cell.text.strip()  # Lưu điểm UDP

    # Chỉ trả về thông tin nếu có cả TCP và UDP
    return student_info if student_info['tcp_score'] and student_info['udp_score'] else None

def main():
  with open('student_scores.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Ghi tiêu đề cột
    writer.writerow(['Mã SV', 'Họ tên', 'Lớp', 'Nhóm môn học', 'Điểm TCP', 'Điểm UDP'])

    # Duyệt qua từng mã sinh viên để cào điểm
    for student_id in student_ids:
      student_info = fetch_student_scores(student_id)  # Cào điểm cho sinh viên
      if student_info:  # Kiểm tra nếu có thông tin
        # Ghi thông tin sinh viên vào file CSV
        writer.writerow([student_id, student_info['name'], student_info['class'], student_info['subject_group'], student_info['tcp_score'], student_info['udp_score']])
        # In thông tin ra màn hình ngay lập tức
        print(f"Đã cào: {student_info['name']} - Mã SV: {student_id} - Lớp: {student_info['class']} - Nhóm môn học: {student_info['subject_group']} - Điểm TCP: {student_info['tcp_score']} - Điểm UDP: {student_info['udp_score']}")
# Chạy hàm chính
if __name__ == "__main__":
    main()

Đã cào: Nguyễn Tuấn Anh - Mã SV: B21DCCN008 - Lớp: D21CNPM4 - Nhóm môn học: 5 - Điểm TCP: 4 - Điểm UDP: 9
Đã cào: Phạm Viết Tài Anh - Mã SV: B21DCCN012 - Lớp: D21HTTT6 - Nhóm môn học: 6 - Điểm TCP: 6 - Điểm UDP: 6
Đã cào: Tạ Xuân Bách - Mã SV: B21DCCN015 - Lớp: E21CNPM1 - Nhóm môn học: 1 - Điểm TCP: 1 - Điểm UDP: 1
Đã cào: Trần Công Bách - Mã SV: B21DCCN016 - Lớp: E21CNPM1 - Nhóm môn học: 1 - Điểm TCP: 6 - Điểm UDP: 9
Đã cào: Tạ Văn Chiến - Mã SV: B21DCCN021 - Lớp: D21HTTT4 - Nhóm môn học: 4 - Điểm TCP: 1 - Điểm UDP: 1
Đã cào: Đỗ Thành Công - Mã SV: B21DCCN022 - Lớp: D21CNPM5 - Nhóm môn học: 7 - Điểm TCP: 4 - Điểm UDP: 6
Đã cào: Hồ Trọng Cường - Mã SV: B21DCCN023 - Lớp: D21HTTT5 - Nhóm môn học: 8 - Điểm TCP: 0 - Điểm UDP: ......
Đã cào: Bùi Xuân Đang - Mã SV: B21DCCN026 - Lớp: D21CNPM1 - Nhóm môn học: 4 - Điểm TCP: 6 - Điểm UDP: 8
Đã cào: Lê Quang Đạt - Mã SV: B21DCCN028 - Lớp: D21HTTT2 - Nhóm môn học: 3 - Điểm TCP: 1 - Điểm UDP: 1
Đã cào: Lưu Hữu Đạt - Mã SV: B21DCCN029 - Lớp: D21HTTT